# Disaster Preparedness Planner  
**Use Case:** Disaster Preparedness Planner  
**Complex Function Integrated:** Risk Assessment Engine  

This notebook implements a disaster-preparedness planning tool using:  
- Free weather APIs (Open-Meteo)  
- Geocoding API (maps.co - free)  
- Primitive functions  
- Complex function: Risk Assessment Engine  
- A final composed workflow that generates a personalized preparedness plan  

# Imports

In [5]:
import requests
import json
from datetime import datetime, timedelta
from pprint import pprint

In [86]:
import json

print("Loading policy coverage JSON...")

with open(r"C:\Users\Gouthum\Downloads\Assisto Technologies Inc\data\sample_policy_coverage.json", "r") as f:
    policy_data = json.load(f)

policy_data

Loading policy coverage JSON...


{'policy_name': 'Standard Home Insurance',
 'coverage': {'fire_damage': True,
  'storm_damage': True,
  'flood_damage': False,
  'theft': True,
  'personal_property': True,
  'temporary_accommodation': True},
 'exclusions': ['Earthquake damage',
  'Wear and tear',
  'Negligence-related loss'],
 'policy_notes': 'Coverage varies for specific disasters. Review your policy document for detailed terms and conditions.'}

# Constants

In [8]:
OPEN_METEO_URL = "https://api.open-meteo.com/v1/forecast"

DEFAULT_WEATHER_PARAMS = {
    "hourly": "temperature_2m,relativehumidity_2m,precipitation",
    "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max",
    "timezone": "auto"
}

{'hourly': 'temperature_2m,relativehumidity_2m,precipitation',
 'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max',
 'timezone': 'auto'}

##  Testing the Open-Meteo Weather API  
This cell sends a real API request to Open-Meteo using Udupi's latitude and longitude.  
It retrieves daily weather forecasts including temperature, rainfall, and wind speed.  
This validates that our `callAPI()` logic and the selected weather endpoint work correctly.


In [10]:
import requests
from datetime import datetime, timedelta

def test_api_call():
    # Example: Udupi coordinates
    lat = 13.34
    lon = 74.74

    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max",
        "timezone": "auto",
        "start_date": datetime.utcnow().date().isoformat(),
        "end_date": (datetime.utcnow().date() + timedelta(days=2)).isoformat()
    }

    response = requests.get(OPEN_METEO_URL, params=params)
    print("API Status Code:", response.status_code)

    data = response.json()
    return data


# Run test
weather_data = test_api_call()
weather_data

API Status Code: 200


{'latitude': 13.375,
 'longitude': 74.75,
 'generationtime_ms': 0.06091594696044922,
 'utc_offset_seconds': 19800,
 'timezone': 'Asia/Kolkata',
 'timezone_abbreviation': 'GMT+5:30',
 'elevation': 27.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'precipitation_sum': 'mm',
  'windspeed_10m_max': 'km/h'},
 'daily': {'time': ['2025-12-10', '2025-12-11', '2025-12-12'],
  'temperature_2m_max': [30.0, 30.4, 29.3],
  'temperature_2m_min': [26.9, 26.7, 25.8],
  'precipitation_sum': [0.0, 0.0, 0.0],
  'windspeed_10m_max': [21.2, 21.9, 19.4]}}

In [96]:
import requests
from datetime import datetime, timedelta

def test_api_call():
    # Example: Bengaluru coordinates
    lat = 12.97194 
    lon = 77.59369

    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max",
        "timezone": "auto",
        "start_date": datetime.utcnow().date().isoformat(),
        "end_date": (datetime.utcnow().date() + timedelta(days=2)).isoformat()
    }

    response = requests.get(OPEN_METEO_URL, params=params)
    print("API Status Code:", response.status_code)

    data = response.json()
    return data


# Run test
weather_data = test_api_call()
weather_data

API Status Code: 200


{'latitude': 13.0,
 'longitude': 77.625,
 'generationtime_ms': 0.07915496826171875,
 'utc_offset_seconds': 19800,
 'timezone': 'Asia/Kolkata',
 'timezone_abbreviation': 'GMT+5:30',
 'elevation': 918.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'precipitation_sum': 'mm',
  'windspeed_10m_max': 'km/h'},
 'daily': {'time': ['2025-12-12', '2025-12-13', '2025-12-14'],
  'temperature_2m_max': [26.2, 26.6, 26.3],
  'temperature_2m_min': [12.8, 13.7, 13.0],
  'precipitation_sum': [0.0, 0.0, 0.0],
  'windspeed_10m_max': [11.3, 10.5, 12.8]}}

##  Testing the Geocoding API(Maps.co) (City → Latitude, Longitude)
This cell tests the free Maps.co Geocoding API, which converts a city name (e.g., "Udupi") into precise geographic coordinates.  
These coordinates are required to call the Open-Meteo weather API in later steps.

In [12]:
import requests

def geocode_city(city_name):
    """
    Convert a city name into (latitude, longitude) using the free Nominatim API.
    No API key required.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": city_name,
        "format": "json",
        "limit": 1
    }

    headers = {
        "User-Agent": "DisasterPreparednessPlanner/1.0"
    }

    response = requests.get(url, params=params, headers=headers, timeout=10)
    response.raise_for_status()

    results = response.json()
    if not results:
        raise ValueError(f"Could not geocode location: {city_name}")

    lat = float(results[0]["lat"])
    lon = float(results[0]["lon"])

    return lat, lon


#  Example test
lat, lon = geocode_city("Udupi")
print("Latitude:", lat, "Longitude:", lon)

Latitude: 13.5269784 Longitude: 74.8731879


In [97]:
import requests

def geocode_city(city_name):
    """
    Convert a city name into (latitude, longitude) using the free Nominatim API.
    No API key required.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": city_name,
        "format": "json",
        "limit": 1
    }

    headers = {
        "User-Agent": "DisasterPreparednessPlanner/1.0"
    }

    response = requests.get(url, params=params, headers=headers, timeout=10)
    response.raise_for_status()

    results = response.json()
    if not results:
        raise ValueError(f"Could not geocode location: {city_name}")

    lat = float(results[0]["lat"])
    lon = float(results[0]["lon"])

    return lat, lon


#  Example test
lat, lon = geocode_city("Bengaluru")
print("Latitude:", lat, "Longitude:", lon)

Latitude: 12.9767936 Longitude: 77.590082


## Defining Risk Mapping and Preparedness Templates  
This cell defines the disaster risk categories and their corresponding thresholds,  
as well as the emergency preparedness templates associated with each risk type.

These mappings are used by the `compare()` and `createWorkflow()` primitive functions  
to determine which disasters are relevant for a location and to generate a personalized  
preparedness plan for the user.

Additional risks such as *Landslide*, *Lightning/Thunderstorm*, and *Cold Wave* are also included  
to make the system more comprehensive and realistic.


# Risk Mapping + Preparedness Templates

In [13]:
risk_mapping = {
    "flood": {
        "threshold": {"precipitation_sum_mm": 50},
        "label": "Flood Risk"
    },
    "storm": {
        "threshold": {"wind_speed_ms": 15},
        "label": "Storm Risk"
    },
    "heatwave": {
        "threshold": {"temperature_c": 40},
        "label": "Heatwave Risk"
    },
    "landslide": {
        "threshold": {"precipitation_sum_mm": 80},
        "label": "Landslide Risk"
    },
    "lightning": {
        "threshold": {"wind_speed_ms": 20},
        "label": "Lightning & Thunderstorm Risk"
    },
    "coldwave": {
        "threshold": {"temperature_c_min": 5},
        "label": "Cold Wave Risk"
    }
}

preparedness_templates = {
    "General": [
        "Keep important documents in waterproof bags.",
        "Maintain a small emergency medical kit.",
        "Know the nearest shelter and evacuation route.",
    ],

    "Flood Risk": [
        "Move valuables to higher shelves.",
        "Prepare extra drinking water and dry food.",
        "Turn off electricity if flooding begins.",
        "Have contact details of local rescue teams."
    ],

    "Storm Risk": [
        "Secure outdoor furniture.",
        "Charge your devices and power banks.",
        "Stay indoors and avoid travel during storm warnings."
    ],

    "Heatwave Risk": [
        "Stay hydrated and avoid going out during peak afternoon.",
        "Check on elderly and children.",
        "Store oral rehydration salts and water."
    ],

    "Landslide Risk": [
        "Avoid travel near steep slopes during heavy rains.",
        "Identify safe alternate routes.",
        "Stay updated with local emergency alerts."
    ],

    "Lightning & Thunderstorm Risk": [
        "Unplug electrical appliances during thunderstorms.",
        "Stay indoors, avoid open areas.",
        "Do not stand under tall trees during lightning."
    ],

    "Cold Wave Risk": [
        "Use proper warm clothing and blankets.",
        "Avoid faulty or unsafe heating equipment.",
        "Provide extra care for elderly, children and pets."
    ]
}

# Primitive Function: extractData

In [20]:
def extract_data(user_input, fields):
    """
    Extract required fields from user_input (a dict).
    Returns a dict with requested keys present (None if missing).
    """
    return {k: user_input.get(k) for k in fields}

In [101]:
import re

def extract_data(user_input, fields):
    """
    Smarter NLP-based extractor.
    Extracts:
        • All locations mentioned (cities)
        • Age mentioned in text ("I am 26 years old")
        • History text as-is
    """

    result = {}

    text_location = user_input.get("location", "") or ""
    text_age = user_input.get("age", "") or ""
    text_history = user_input.get("history", "") or ""

    # ------------------------
    # 1. Extract ALL cities
    # ------------------------
    city_list = [
        "udupi","mumbai","delhi","bengaluru","bangalore","Kundapura","mysore","mangaluru",
        "chennai","hyderabad","pune","ahmedabad","kolkata","jaipur","lucknow",
        "surat","indore","bhopal","coimbatore","madurai","patna","ranchi",
        "visakhapatnam","kochi","trivandrum"
    ]

    # Find all cities in the location text
    found_cities = []
    for city in city_list:
        if city in text_location.lower():
            found_cities.append(city.title())

    # If no recognizable cities → fallback
    if found_cities:
        result["location"] = found_cities
    else:
        result["location"] = [text_location.strip()]

    # ------------------------
    # 2. Extract AGE from text
    # ------------------------
    age_pattern = r"\b(\d{1,2})\s*(years old|yrs old|year old|yrs)\b"
    match = re.search(age_pattern, text_age.lower())

    if match:
        result["age"] = int(match.group(1))
    else:
        # fallback if age is given directly as number
        number_only = re.findall(r"\b\d{1,2}\b", text_age)
        result["age"] = int(number_only[0]) if number_only else None

    # ------------------------
    # 3. HISTORY (return full text)
    # ------------------------
    result["history"] = text_history.strip()

    # Filter output to requested fields only
    return {field: result.get(field) for field in fields}


In [102]:
sample = {
    "location": "I stay in Udupi,Its My hometown eariler I was located in bengaluru, after that we went to mysore now im in udupi",
    "age": "I am 26 years old",
    "history": "My house flooded last year during the monsoon."
}

result = extract_data(sample, ["location", "age", "history"])
print(result)


{'location': ['Udupi', 'Bengaluru', 'Mysore'], 'age': 26, 'history': 'My house flooded last year during the monsoon.'}


In [21]:
#Testing 
sample = {
    "location": "Udupi",
    "policy": "Home Insurance",
    "history": "flood last year"
}

result = extract_data(sample, ["location", "history"])
print(result)

{'location': 'Udupi', 'history': 'flood last year'}


# Primitive Function: classifyText

In [19]:
def classify_text(history_text):
    """
    Very simple rule-based classifier to convert user history into a risk level:
    returns one of: 'low', 'medium', 'high'
    (In a real LLM or ML approach, we'd use an LLM or model.)
    """
    if not history_text:
        return "low"
    text = history_text.lower()
    score = 0
    if any(w in text for w in ["flood", "inundation", "overflow", "river"]):
        score += 2
    if any(w in text for w in ["storm", "cyclone", "wind", "hail"]):
        score += 1
    if any(w in text for w in ["frequent", "frequently", "repeated", "recent"]):
        score += 1
    if "evacuated" in text or "major damage" in text:
        score += 3

    if score >= 3:
        return "high"
    elif score >= 1:
        return "medium"
    else:
        return "low"

In [17]:
# Test classify_text()

print(classify_text("My house was flooded last year"))
print(classify_text("There were strong winds and a storm recently"))
print(classify_text("We were evacuated due to heavy rains"))
print(classify_text("No issues in my area"))
print(classify_text(""))

medium
medium
high
low
low


# Primitive Function: classifyText

In [103]:
def classify_text(history_text: str):
    """
    Realistic disaster-history classifier.
    
    Converts user history text into a risk rating:
        'low', 'medium', 'high', 'critical'

    Uses weighted keyword + severity patterns based on real-world scenarios.
    """
    if not history_text or history_text.strip() == "":
        return "low"

    text = history_text.lower()
    score = 0

    # ---------------------------
    # Flood / water-related risks
    # ---------------------------
    flood_words = ["flood", "water entered", "inundation", "overflow", 
                   "dam release", "heavy rain", "monsoon damage"]
    for w in flood_words:
        if w in text:
            score += 2

    # ---------------------------
    # Cyclone / storm / wind risks
    # ---------------------------
    storm_words = ["cyclone", "storm", "strong winds", "hailstorm", 
                   "wind damage", "tree fell", "roof damaged"]
    for w in storm_words:
        if w in text:
            score += 2

    # ---------------------------
    # Landslide risks
    # ---------------------------
    landslide_words = ["landslide", "mudslide", "hill collapse", "soil erosion"]
    for w in landslide_words:
        if w in text:
            score += 3

    # ---------------------------
    # Heat-related risks
    # ---------------------------
    heat_words = ["heatwave", "extreme heat", "very high temperature"]
    for w in heat_words:
        if w in text:
            score += 1

    # ---------------------------
    # Infrastructure indicators
    # ---------------------------
    infra_words = ["power outage", "no electricity", "roads blocked", 
                   "bridge collapsed", "water logging", "house leakage"]
    for w in infra_words:
        if w in text:
            score += 1

    # ---------------------------
    # Evacuation / severe damage
    # ---------------------------
    if "evacuated" in text:
        score += 4

    if "major damage" in text or "severe damage" in text:
        score += 3

    if "lost property" in text or "property damage" in text:
        score += 2

    # ---------------------------
    # Frequency indicators
    # ---------------------------
    if any(w in text for w in ["frequent", "repeated", "every year", "commonly happens"]):
        score += 2

    # ---------------------------
    # Final scoring thresholds
    # ---------------------------
    if score >= 9:
        return "critical"
    elif score >= 5:
        return "high"
    elif score >= 2:
        return "medium"
    else:
        return "low"

# REAL-WORLD TEST CASES

In [104]:
print(classify_text("My house flooded last year and water entered the ground floor."))
print(classify_text("Cyclone hit our area and many roofs were damaged."))
print(classify_text("We were evacuated due to a landslide in the nearby hill."))
print(classify_text("Power outage and road blockage happened due to heavy rain."))
print(classify_text("We experience heatwaves every year."))
print(classify_text("No major issues have happened in my area."))
print(classify_text(""))

medium
medium
high
medium
medium
low
low


# Primitive function: compare (weather risk and  thresholds)

In [22]:
def compare_weather_to_thresholds(daily_data):
    """
    Compare daily_data (dict returned from Open-Meteo daily) to thresholds in risk_mapping.
    Returns a dict of found risks and basic metrics.
    """
    risks_found = {}
    # daily_data contains arrays for keys like precipitation_sum, temperature_2m_max
    # We'll check max of precipitation_sum across days to decide flood risk, etc.
    precip = daily_data.get("precipitation_sum", [])
    temp_max = daily_data.get("temperature_2m_max", [])
    wind_max = daily_data.get("windspeed_10m_max", [])

    max_precip = max(precip) if precip else 0
    max_temp = max(temp_max) if temp_max else None
    max_wind = max(wind_max) if wind_max else None

    # Compare to thresholds (simple)
    if max_precip and max_precip >= risk_mapping["flood"]["threshold"]["precipitation_sum_mm"]:
        risks_found["flood"] = {"label": risk_mapping["flood"]["label"], "metric": max_precip}
    if max_wind and max_wind >= risk_mapping["storm"]["threshold"]["wind_speed_ms"]:
        risks_found["storm"] = {"label": risk_mapping["storm"]["label"], "metric": max_wind}
    if max_temp and max_temp >= risk_mapping["heatwave"]["threshold"]["temperature_c"]:
        risks_found["heatwave"] = {"label": risk_mapping["heatwave"]["label"], "metric": max_temp}

    return {
        "max_precip_mm": max_precip,
        "max_temp_c": max_temp,
        "max_wind_ms": max_wind,
        "risks_found": risks_found
    }

# Test compare_weather_to_thresholds()

In [23]:
# 1. Get sample weather data using My  API
lat, lon = 13.34, 74.74   # Udupi coordinates

daily_data = call_api_weather(lat, lon)

print("Daily Weather Data Received:")
print(daily_data)

# 2. Pass into compare function
comparison_result = compare_weather_to_thresholds(daily_data)

print("\nComparison Result (Risk Detection):")
print(comparison_result)

Daily Weather Data Received:
{'time': ['2025-12-10', '2025-12-11', '2025-12-12'], 'temperature_2m_max': [30.0, 30.4, 29.3], 'temperature_2m_min': [26.9, 26.7, 25.8], 'precipitation_sum': [0.0, 0.0, 0.0], 'windspeed_10m_max': [21.2, 21.9, 19.4]}

Comparison Result (Risk Detection):
{'max_precip_mm': 0.0, 'max_temp_c': 30.4, 'max_wind_ms': 21.9, 'risks_found': {'storm': {'label': 'Storm Risk', 'metric': 21.9}}}


# Primitive function: aggregate (combine risk components)

In [24]:
def aggregate_scores(history_risk, weather_risks):
    """
    Aggregate simple numeric scores:
    - history_risk: 'low' -> 0.1, 'medium' -> 0.5, 'high' -> 0.9
    - weather_risks: dict of risks_found -> add weights
    Returns an aggregate [0..1] score and textual level.
    """
    mapping = {"low": 0.1, "medium": 0.5, "high": 0.9}
    hist_score = mapping.get(history_risk, 0.1)

    weather_score = 0.0
    for k in weather_risks.get("risks_found", {}):
        if k == "flood":
            weather_score += 0.6
        elif k == "storm":
            weather_score += 0.4
        elif k == "heatwave":
            weather_score += 0.3

    # cap
    weather_score = min(weather_score, 1.0)

    # simple average-like aggregator
    combined = (hist_score + weather_score) / 2.0
    
    # normalize to 0..1
    combined = max(0.0, min(1.0, combined))

    if combined >= 0.7:
        level = "High"
    elif combined >= 0.4:
        level = "Medium"
    else:
        level = "Low"

    return {"score": round(combined, 2), "level": level, "history_score": hist_score, "weather_score": weather_score}

# Test Cases for aggregate_scores()

In [25]:
# Case 1: Low history, no weather risks
history_risk_1 = "low"
weather_risks_1 = {"risks_found": {}}

print("Test 1 (Low):")
print(aggregate_scores(history_risk_1, weather_risks_1))
print("\n")


# Case 2: Medium history, storm detected
history_risk_2 = "medium"
weather_risks_2 = {
    "risks_found": {
        "storm": {"label": "Storm Risk", "metric": 22.0}
    }
}

print("Test 2 (Medium):")
print(aggregate_scores(history_risk_2, weather_risks_2))
print("\n")


# Case 3: High history, flood + heatwave detected
history_risk_3 = "high"
weather_risks_3 = {
    "risks_found": {
        "flood": {"label": "Flood Risk", "metric": 60},
        "heatwave": {"label": "Heatwave Risk", "metric": 42}
    }
}

print("Test 3 (High):")
print(aggregate_scores(history_risk_3, weather_risks_3))


Test 1 (Low):
{'score': 0.05, 'level': 'Low', 'history_score': 0.1, 'weather_score': 0.0}


Test 2 (Medium):
{'score': 0.45, 'level': 'Medium', 'history_score': 0.5, 'weather_score': 0.4}


Test 3 (High):
{'score': 0.9, 'level': 'High', 'history_score': 0.9, 'weather_score': 0.8999999999999999}


# Primitive function: Realistic aggregate (combine risk components)

In [106]:
def age_to_score(age):
    """Convert age to vulnerability score"""
    if age is None:
        return 0.3  # default adult
    if age < 12:
        return 0.8
    elif age <= 60:
        return 0.3
    else:
        return 0.9


def aggregate_scores(age, history_risk, weather_risks):
    """
    Aggregate ageScore, historyScore, weatherScore using average
    """

    # -----------------------------
    # 1. ageScore
    # -----------------------------
    age_score = age_to_score(age)

    # -----------------------------
    # 2. historyScore
    # -----------------------------
    history_mapping = {"low": 0.1, "medium": 0.5, "high": 0.9}
    history_score = history_mapping.get(history_risk, 0.1)

    # -----------------------------
    # 3. weatherScore
    # -----------------------------
    weather_score = 0.0
    for risk in weather_risks.get("risks_found", {}):
        if risk == "flood":
            weather_score += 0.6
        elif risk == "storm":
            weather_score += 0.4
        elif risk == "heatwave":
            weather_score += 0.3

    weather_score = min(weather_score, 1.0)

    # -----------------------------
    # 4. aggregate (average)
    # -----------------------------
    combined = (age_score + history_score + weather_score) / 3
    combined = round(min(max(combined, 0), 1), 2)

    # -----------------------------
    # 5. Risk level
    # -----------------------------
    if combined >= 0.7:
        level = "High"
    elif combined >= 0.4:
        level = "Medium"
    else:
        level = "Low"

    return {
        "score": combined,
        "level": level,
        "age_score": age_score,
        "history_score": history_score,
        "weather_score": weather_score
    }


# REAL-WORLD TEST CASES (Required by assessment)
🧪 Test 1 — Adult, no disasters, calm weather

In [107]:
print(aggregate_scores(
    age=26,
    history_risk="low",
    weather_risks={"risks_found": {}}
))


{'score': 0.13, 'level': 'Low', 'age_score': 0.3, 'history_score': 0.1, 'weather_score': 0.0}


##### 🧪 Test 2 — Elderly + storm

In [108]:
print(aggregate_scores(
    age=68,
    history_risk="medium",
    weather_risks={"risks_found": {"storm": {"metric": 22}}}
))



{'score': 0.6, 'level': 'Medium', 'age_score': 0.9, 'history_score': 0.5, 'weather_score': 0.4}


##### 🧪 Test 3 — Child + flood + heatwave

In [109]:
print(aggregate_scores(
    age=8,
    history_risk="high",
    weather_risks={
        "risks_found": {
            "flood": {"metric": 60},
            "heatwave": {"metric": 42}
        }
    }
))

{'score': 0.87, 'level': 'High', 'age_score': 0.8, 'history_score': 0.9, 'weather_score': 0.8999999999999999}


# Primitive function: guide_user (simulated prompts)

In [26]:
def guide_user_interaction(simulated_inputs=None):
    """
    Simulated conversational guide to collect missing data.
    If simulated_inputs dict provided, use it. Otherwise uses input() for interactive mode.
    Returns a user_input dict with 'location', 'policy', 'history' fields.
    """
    if simulated_inputs:
        # assume keys exist
        return {
            "location": simulated_inputs.get("location"),
            "policy": simulated_inputs.get("policy"),
            "history": simulated_inputs.get("history")
        }
    # interactive fallback
    loc = input("Enter your city or coordinates (city name or 'lat,lon'): ").strip()
    policy = input("Which policy coverage do you have (e.g., 'Standard Home Insurance')? ").strip()
    history = input("Any recent incidents/history (short)? ").strip()
    return {"location": loc, "policy": policy, "history": history}

# Test guide_user_interaction with simulated input

In [27]:
test_input = {
    "location": "Udupi",
    "policy": "Standard Home Insurance",
    "history": "My area had flooding last year"
}

result = guide_user_interaction(test_input)
print(result)

{'location': 'Udupi', 'policy': 'Standard Home Insurance', 'history': 'My area had flooding last year'}


# REAL Primitive function: guide_user (simulated prompts)

In [110]:
# src/primitive_functions/guide_user.py

def guide_user_interaction(simulated_inputs=None):
    """
    Guide the user to provide required inputs for risk assessment.

    Responsibility:
    - ONLY collect user inputs
    - NO API calls
    - NO risk logic

    Returns:
    {
        "location": str,
        "age": int | None,
        "policy": str,
        "history": str
    }
    """

    # -----------------------------
    # Simulated / programmatic input
    # -----------------------------
    if simulated_inputs:
        return {
            "location": simulated_inputs.get("location"),
            "age": simulated_inputs.get("age"),
            "policy": simulated_inputs.get("policy", "Standard Home Insurance"),
            "history": simulated_inputs.get("history", "")
        }

    # -----------------------------
    # Interactive CLI fallback
    # -----------------------------
    print("\n--- Disaster Preparedness Input ---")

    # Location
    while True:
        location = input("Enter your city or coordinates (city or 'lat,lon'): ").strip()
        if location:
            break
        print("❗ Location is required.")

    # Age (optional but useful)
    age_input = input("Enter your age (optional): ").strip()
    try:
        age = int(age_input) if age_input else None
    except ValueError:
        age = None

    # Policy
    policy = input(
        "Enter your policy coverage (default: Standard Home Insurance): "
    ).strip()
    if not policy:
        policy = "Standard Home Insurance"

    # History
    history = input(
        "Any past incidents? (e.g., flooding, storms, evacuation): "
    ).strip()

    return {
        "location": location,
        "age": age,
        "policy": policy,
        "history": history
    }


# TEST EXAMPLES

In [111]:
test_input = {
    "location": "Udupi",
    "age": 26,
    "policy": "Standard Home Insurance",
    "history": "My area had flooding last year"
}

result = guide_user_interaction(test_input)
print(result)

{'location': 'Udupi', 'age': 26, 'policy': 'Standard Home Insurance', 'history': 'My area had flooding last year'}


# Helper: geocode simple city -> lat/lon (uses free geocode.maps.co)

In [30]:
def geocode_city(city_name):
    """
    Convert a city name into (latitude, longitude) using the free Nominatim API.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": city_name,
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "DisasterPreparednessPlanner"
    }
    resp = requests.get(url, params=params, headers=headers, timeout=10)
    resp.raise_for_status()
    results = resp.json()
    if not results:
        raise ValueError("Could not geocode location.")
    return float(results[0]["lat"]), float(results[0]["lon"])

In [33]:
# Test the geocode_city() function
lat, lon = geocode_city("mumbai")
print("Latitude:", lat)
print("Longitude:", lon)

Latitude: 19.054999
Longitude: 72.8692035


# create_workflow: build preparedness plan from risks + templates + policy

In [35]:
def create_preparedness_workflow(location, policy, risks_found):
    """
    Using the risks_found dict, pick templates and create a step-by-step plan.
    Return a dict with plan steps and helpful notes.
    """
    plan = []
    # Always add general tips
    plan.extend(preparedness_templates.get("General", []))

    if not risks_found:
        plan.append("No immediate weather-based risks detected. Maintain general preparedness.")
    else:
        # for each detected risk, add matching templates
        for risk_key, info in risks_found.items():
            label = info["label"]
            plan.append(f"--- {label} (metric: {info.get('metric')}) ---")
            plan.extend(preparedness_templates.get(label, []))

    # Add policy-specific note (very simple)
    plan.append(f"Policy note: Your recorded policy is '{policy}'. Check policy coverages for relevant damages (flood/storm) and save insurer contact.")
    return {
        "location": location,
        "policy": policy,
        "plan": plan,
        "generated_at": datetime.utcnow().isoformat()
    }

# Test the create_preparedness_workflow function

In [36]:
test_location = "Udupi"
test_policy = "Standard Home Insurance"

# Simulated risks_found example (normally comes from compare_weather_to_thresholds)
test_risks_found = {
    "storm": {"label": "Storm Risk", "metric": 22.5},
    "heatwave": {"label": "Heatwave Risk", "metric": 41.2}
}

workflow = create_preparedness_workflow(test_location, test_policy, test_risks_found)

workflow


{'location': 'Udupi',
 'policy': 'Standard Home Insurance',
 'plan': ['Keep important documents in waterproof bags.',
  'Maintain a small emergency medical kit.',
  'Know the nearest shelter and evacuation route.',
  '--- Storm Risk (metric: 22.5) ---',
  'Secure outdoor furniture.',
  'Charge your devices and power banks.',
  'Stay indoors and avoid travel during storm warnings.',
  '--- Heatwave Risk (metric: 41.2) ---',
  'Stay hydrated and avoid going out during peak afternoon.',
  'Check on elderly and children.',
  'Store oral rehydration salts and water.',
  "Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact."],
 'generated_at': '2025-12-10T20:57:32.725612'}

# Real create_workflow

In [112]:
from datetime import datetime

# Expanded preparedness templates
preparedness_templates = {
    "General": [
        "Keep important documents in waterproof bags.",
        "Maintain a basic emergency medical kit.",
        "Know the nearest shelter and evacuation route.",
        "Save emergency contact numbers on your phone.",
        "Keep a torch, batteries, and power bank ready."
    ],

    "Flood Risk": [
        "Move valuables and electrical items to higher levels.",
        "Avoid driving or walking through flooded roads.",
        "Switch off main electricity supply during flooding.",
        "Keep sandbags or barriers ready near entrances."
    ],

    "Storm Risk": [
        "Secure outdoor furniture and loose objects.",
        "Charge phones and power banks before the storm.",
        "Stay indoors and avoid unnecessary travel.",
        "Park vehicles away from trees and power lines."
    ],

    "Heatwave Risk": [
        "Stay hydrated and drink water frequently.",
        "Avoid outdoor activities during peak afternoon hours.",
        "Check on elderly people, children, and pets.",
        "Keep ORS packets and light food available."
    ],

    "Cyclone Risk": [
        "Identify cyclone shelters in your area.",
        "Reinforce doors and windows.",
        "Prepare an emergency bag with essentials.",
        "Follow official evacuation orders immediately."
    ],

    "Landslide Risk": [
        "Avoid staying near slopes during heavy rainfall.",
        "Watch for cracks or soil movement near your home.",
        "Evacuate early if authorities issue warnings."
    ],

    "Earthquake Risk": [
        "Secure heavy furniture to walls.",
        "Practice Drop–Cover–Hold drills.",
        "Identify safe spots inside your home."
    ],

    "Power Outage": [
        "Keep backup lighting and batteries ready.",
        "Store drinking water in advance.",
        "Avoid opening refrigerators frequently."
    ]
}


def create_preparedness_workflow(location, policy, risks_found):
    """
    Build a preparedness workflow using detected risks,
    regional relevance, and policy coverage.
    """

    plan = []

    # -------------------------
    # General preparedness
    # -------------------------
    plan.extend(preparedness_templates["General"])

    # -------------------------
    # Risk-specific actions
    # -------------------------
    if not risks_found:
        plan.append(
            "No immediate weather-based risks detected. Maintain general preparedness."
        )
    else:
        for risk_key, info in risks_found.items():
            label = info.get("label", risk_key)
            metric = info.get("metric")

            plan.append(f"--- {label} (Observed metric: {metric}) ---")

            # Add matching templates
            if label in preparedness_templates:
                plan.extend(preparedness_templates[label])
            else:
                plan.append(
                    "Follow official advisories and stay alert for updates."
                )

    # -------------------------
    # Policy guidance
    # -------------------------
    plan.append(
        f"Policy Note: Your current policy is '{policy}'. "
        "Review coverage for flood, storm, and structural damage. "
        "Keep insurer contact details accessible."
    )

    # -------------------------
    # Final structured output
    # -------------------------
    return {
        "location": location,
        "policy": policy,
        "plan": plan,
        "generated_at": datetime.utcnow().isoformat(),
        "template_used": list(risks_found.keys()) if risks_found else ["General"]
    }


# TEST THE CASE

In [114]:
test_location = "Udupi"
test_policy = "Standard Home Insurance"

# Simulate ALL detected risks (as if compare() detected them)
test_risks_found = {
    "flood": {"label": "Flood Risk", "metric": 68.0},
    "storm": {"label": "Storm Risk", "metric": 24.5},
    "heatwave": {"label": "Heatwave Risk", "metric": 42.1},
    "cyclone": {"label": "Cyclone Risk", "metric": "Category 2"},
    "landslide": {"label": "Landslide Risk", "metric": "High rainfall"},
    "earthquake": {"label": "Earthquake Risk", "metric": "Zone III"},
    "power_outage": {"label": "Power Outage", "metric": "12 hours"}
}

workflow = create_preparedness_workflow(
    test_location,
    test_policy,
    test_risks_found
)

workflow


{'location': 'Udupi',
 'policy': 'Standard Home Insurance',
 'plan': ['Keep important documents in waterproof bags.',
  'Maintain a basic emergency medical kit.',
  'Know the nearest shelter and evacuation route.',
  'Save emergency contact numbers on your phone.',
  'Keep a torch, batteries, and power bank ready.',
  '--- Flood Risk (Observed metric: 68.0) ---',
  'Move valuables and electrical items to higher levels.',
  'Avoid driving or walking through flooded roads.',
  'Switch off main electricity supply during flooding.',
  'Keep sandbags or barriers ready near entrances.',
  '--- Storm Risk (Observed metric: 24.5) ---',
  'Secure outdoor furniture and loose objects.',
  'Charge phones and power banks before the storm.',
  'Stay indoors and avoid unnecessary travel.',
  'Park vehicles away from trees and power lines.',
  '--- Heatwave Risk (Observed metric: 42.1) ---',
  'Stay hydrated and drink water frequently.',
  'Avoid outdoor activities during peak afternoon hours.',
  'Ch

# display_information: pretty print the plan

In [37]:
def display_plan(plan_obj):
    """
    Nicely print out the preparedness plan.
    """
    print("=== Disaster Preparedness Plan ===")
    print("Location:", plan_obj["location"])
    print("Policy:", plan_obj["policy"])
    print("Generated at (UTC):", plan_obj["generated_at"])
    print("\nPlan steps:")
    for i, step in enumerate(plan_obj["plan"], 1):
        print(f"{i}. {step}")
    print("\n(End of plan)")

# Test display_plan()

In [38]:
sample_plan = {
    "location": "Udupi",
    "policy": "Standard Home Insurance",
    "generated_at": "2025-12-11T12:00:00Z",
    "plan": [
        "Keep important documents in waterproof bags.",
        "Maintain a small emergency medical kit.",
        "Know the nearest shelter and evacuation route.",
        "--- Storm Risk (metric: 22.1) ---",
        "Secure outdoor furniture.",
        "Charge your devices and power banks.",
        "Stay indoors and avoid travel during storm warnings.",
        "Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact."
    ]
}

display_plan(sample_plan)

=== Disaster Preparedness Plan ===
Location: Udupi
Policy: Standard Home Insurance
Generated at (UTC): 2025-12-11T12:00:00Z

Plan steps:
1. Keep important documents in waterproof bags.
2. Maintain a small emergency medical kit.
3. Know the nearest shelter and evacuation route.
4. --- Storm Risk (metric: 22.1) ---
5. Secure outdoor furniture.
6. Charge your devices and power banks.
7. Stay indoors and avoid travel during storm warnings.
8. Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact.

(End of plan)


# Updated display_information: pretty print the plan

In [116]:
def display_plan(plan_obj):
    """
    Display a real-life disaster preparedness report.
    """

    print("\n" + "=" * 60)
    print("🌪️ DISASTER PREPAREDNESS REPORT")
    print("=" * 60)

    # -------------------------
    # Basic Info
    # -------------------------
    print(f"📍 Location        : {plan_obj.get('location')}")
    print(f"🛡️ Policy          : {plan_obj.get('policy')}")
    print(f"🕒 Generated (UTC) : {plan_obj.get('generated_at')}")

    # -------------------------
    # Risk Summary (if exists)
    # -------------------------
    if "risk_summary" in plan_obj:
        rs = plan_obj["risk_summary"]
        print("\n⚠️ RISK SUMMARY")
        print("-" * 40)
        print(f"Overall Level : {rs.get('level')}")
        print(f"Risk Score    : {rs.get('score')}")
        print(f"History Score : {rs.get('history_score')}")
        print(f"Weather Score : {rs.get('weather_score')}")

    # -------------------------
    # Preparedness Plan
    # -------------------------
    print("\n📋 PREPAREDNESS ACTION PLAN")
    print("-" * 40)

    for i, step in enumerate(plan_obj.get("plan", []), 1):
        if step.startswith("---"):
            # Risk section header
            print(f"\n{step}")
        else:
            print(f"{i}. {step}")

    # -------------------------
    # Template Info
    # -------------------------
    if "template_used" in plan_obj:
        print("\n🧩 Templates Used:")
        for t in plan_obj["template_used"]:
            print(f"- {t}")

    print("\n" + "=" * 60)
    print("✅ END OF REPORT")
    print("=" * 60 + "\n")


# TEST CASE

In [119]:
sample_plan = {
    "location": "Udupi",
    "policy": "Standard Home Insurance",
    "generated_at": "2025-12-11T12:00:00Z",
    "risk_summary": {
        "score": 0.82,
        "level": "High",
        "history_score": 0.9,
        "weather_score": 0.75
    },
    "plan": [
        # ---------------- General Preparedness ----------------
        "Keep important documents in waterproof bags.",
        "Maintain a basic emergency medical kit.",
        "Save emergency contact numbers on your phone.",
        "Know the nearest shelter and evacuation routes.",

        # ---------------- Flood Risk ----------------
        "--- Flood Risk (Observed metric: 65 mm rainfall) ---",
        "Move valuables, documents, and electronics to higher floors.",
        "Switch off main electricity and gas connections if flooding is imminent.",
        "Avoid walking or driving through flooded roads.",
        "Keep sandbags or barriers ready near entrances.",
        "Store drinking water and food in waterproof containers.",
        "Identify nearest elevated safe shelter or evacuation center.",
        "Monitor official flood warnings and rainfall alerts.",
        "Keep emergency helpline numbers accessible.",

        # ---------------- Storm Risk ----------------
        "--- Storm Risk (Observed metric: 22 m/s wind speed) ---",
        "Secure loose outdoor furniture and objects.",
        "Park vehicles away from trees and power lines.",
        "Charge mobile phones, power banks, and emergency radios.",
        "Close and reinforce windows and doors.",
        "Avoid unnecessary travel during storm alerts.",
        "Keep torchlights and spare batteries ready.",
        "Stay indoors until official clearance is announced.",
        "Follow district administration advisories.",

        # ---------------- Heatwave Risk ----------------
        "--- Heatwave Risk (Observed metric: 41 °C) ---",
        "Drink water frequently, even if not thirsty.",
        "Avoid outdoor activities during peak afternoon hours.",
        "Wear light-colored, loose cotton clothing.",
        "Use curtains or shades to reduce indoor heat.",
        "Check on elderly people, children, and pets regularly.",
        "Keep ORS packets or electrolyte drinks ready.",
        "Take cool showers or sponge baths if overheated.",
        "Seek medical help for dizziness or heat exhaustion.",

        # ---------------- Policy Note ----------------
        "Policy Note: Review coverage for flood, storm, and heat-related damages. Keep insurer contact details accessible."
    ],
    "template_used": ["Flood Risk", "Storm Risk", "Heatwave Risk"]
}


display_plan(sample_plan)


🌪️ DISASTER PREPAREDNESS REPORT
📍 Location        : Udupi
🛡️ Policy          : Standard Home Insurance
🕒 Generated (UTC) : 2025-12-11T12:00:00Z

⚠️ RISK SUMMARY
----------------------------------------
Overall Level : High
Risk Score    : 0.82
History Score : 0.9
Weather Score : 0.75

📋 PREPAREDNESS ACTION PLAN
----------------------------------------
1. Keep important documents in waterproof bags.
2. Maintain a basic emergency medical kit.
3. Save emergency contact numbers on your phone.
4. Know the nearest shelter and evacuation routes.

--- Flood Risk (Observed metric: 65 mm rainfall) ---
6. Move valuables, documents, and electronics to higher floors.
7. Switch off main electricity and gas connections if flooding is imminent.
8. Avoid walking or driving through flooded roads.
9. Keep sandbags or barriers ready near entrances.
10. Store drinking water and food in waterproof containers.
11. Identify nearest elevated safe shelter or evacuation center.
12. Monitor official flood warnin

# Complex function: risk_assessment_engine (combines primitives)

In [135]:
# src/risk_assessment_engine.py

from src.primitive_functions.extract_data import extract_data
from src.primitive_functions.classify_text import classify_text
from src.primitive_functions.call_api import call_api_weather, geocode_city
from src.primitive_functions.compare import compare_weather_to_thresholds
from src.primitive_functions.aggregate import aggregate_scores


def risk_assessment_engine(user_input):
    """
    Risk Assessment Engine (ROBUST & SAFE)

    Steps:
    1) extractData
    2) geocode city / parse lat,lon
    3) callAPI (Open-Meteo)
    4) compare thresholds
    5) classifyText
    6) aggregate scores
    """

    # -----------------------------------------------------------
    # 1. extractData
    # -----------------------------------------------------------
    data = extract_data(user_input, ["location", "policy", "history"])

    location = data.get("location")
    policy = data.get("policy")
    history = data.get("history")

    if not location:
        raise ValueError("Location is required")

    # -----------------------------------------------------------
    # 2. Convert location → lat/lon (SAFE & ROBUST)
    # -----------------------------------------------------------
    lat = lon = None

    # Case A: lat,lon string
    if isinstance(location, str) and "," in location:
        try:
            lat, lon = map(float, location.split(","))
        except Exception:
            raise ValueError("Invalid coordinates format. Use 'lat,lon'")

    # Case B: city name
    elif isinstance(location, str):
        coords = geocode_city(location.strip())

        if not coords:
            raise ValueError(
                f"Could not find coordinates for city '{location}'. "
                "Please try another city or provide lat,lon."
            )

        lat, lon = coords

    # Case C: tuple/list
    elif isinstance(location, (list, tuple)) and len(location) == 2:
        lat, lon = location

    else:
        raise ValueError("Location must be a city name or 'lat,lon'")

    # -----------------------------------------------------------
    # 3. callAPI - Open-Meteo weather
    # -----------------------------------------------------------
    daily_weather = call_api_weather(lat, lon, days=3)

    if not daily_weather:
        raise RuntimeError("Weather API returned no data")

    # -----------------------------------------------------------
    # 4. compare - weather thresholds
    # -----------------------------------------------------------
    weather_analysis = compare_weather_to_thresholds(daily_weather)

    # -----------------------------------------------------------
    # 5. classifyText - history risk
    # -----------------------------------------------------------
    history_risk = classify_text(history)

    # -----------------------------------------------------------
    # 6. aggregate - final score
    # -----------------------------------------------------------
    aggregate_result = aggregate_scores(history_risk, weather_analysis)

    # -----------------------------------------------------------
    # Final Output
    # -----------------------------------------------------------
    return {
        "location": location,
        "lat": lat,
        "lon": lon,
        "history_risk": history_risk,
        "weather_analysis": weather_analysis,
        "aggregate": aggregate_result,
        "policy": policy,
    }


In [40]:
# Test input for Risk Assessment Engine
user_input_test = {
    "location": "Udupi",
    "policy": "Standard Home Insurance",
    "history": "There was a storm last year but no flood."
}

result = risk_assessment_engine(user_input_test)
result

{'location': 'Udupi',
 'lat': 13.5269784,
 'lon': 74.8731879,
 'history_risk': 'high',
 'weather_analysis': {'max_precip_mm': 0.0,
  'max_temp_c': 35.6,
  'max_wind_ms': 14.2,
  'risks_found': {}},
 'aggregate': {'score': 0.45,
  'level': 'Medium',
  'history_score': 0.9,
  'weather_score': 0.0},
 'policy': 'Standard Home Insurance'}

# Test Case 1: Coastal Flood-Prone Area (High Risk)

In [136]:
user_input_test_1 = {
    "location": "Udupi",
    "policy": "Standard Home Insurance",
    "history": "My house was flooded during heavy monsoon rains last year and we had to evacuate."
}

result_1 = risk_assessment_engine(user_input_test_1)
result_1

{'location': 'Udupi',
 'lat': 13.5269784,
 'lon': 74.8731879,
 'history_risk': 'medium',
 'weather_analysis': {'max_precip_mm': 0.0,
  'max_temp_c': 34.7,
  'max_wind_ms': 4.17,
  'risks_found': {}},
 'aggregate': {'score': 0.25,
  'level': 'Low',
  'history_score': 0.5,
  'weather_score': 0.0},
 'policy': 'Standard Home Insurance'}

# Test Case 2: Metro City with Heatwave & Storms (Medium Risk)

In [137]:
user_input_test_2 = {
    "location": "Bengaluru",
    "policy": "Apartment Insurance",
    "history": "There were frequent power cuts and strong winds during summer storms, but no major damage."
}

result_2 = risk_assessment_engine(user_input_test_2)
result_2

{'location': 'Bengaluru',
 'lat': 12.9767936,
 'lon': 77.590082,
 'history_risk': 'high',
 'weather_analysis': {'max_precip_mm': 0.0,
  'max_temp_c': 26.3,
  'max_wind_ms': 3.78,
  'risks_found': {}},
 'aggregate': {'score': 0.45,
  'level': 'Medium',
  'history_score': 0.9,
  'weather_score': 0.0},
 'policy': 'Apartment Insurance'}

# Test Case 3: Dry Inland City with Minimal History (Low Risk)

In [138]:
user_input_test_3 = {
    "location": "Jaipur",
    "policy": "Basic Home Insurance",
    "history": "No major disasters so far. Weather is usually dry and stable."
}

result_3 = risk_assessment_engine(user_input_test_3)
result_3

{'location': 'Jaipur',
 'lat': 26.9154576,
 'lon': 75.8189817,
 'history_risk': 'low',
 'weather_analysis': {'max_precip_mm': 0.0,
  'max_temp_c': 28.0,
  'max_wind_ms': 2.19,
  'risks_found': {}},
 'aggregate': {'score': 0.05,
  'level': 'Low',
  'history_score': 0.1,
  'weather_score': 0.0},
 'policy': 'Basic Home Insurance'}

In [88]:
import json
import os

# Set your Windows path
save_path = r"C:\Users\Gouthum\Downloads\Assisto Technologies Inc\example"

# Create folder if not exists
os.makedirs(save_path, exist_ok=True)

# Build full file path
file_path = os.path.join(save_path, "risk_assessment_output.json")

# Save the JSON
with open(file_path, "w") as f:
    json.dump(result, f, indent=2)

print(f"risk_assessment_output.json saved successfully at:\n{file_path}")

risk_assessment_output.json saved successfully at:
C:\Users\Gouthum\Downloads\Assisto Technologies Inc\example\risk_assessment_output.json


This will execute your entire pipeline:

extractData()

classifyText()

callAPI()

compare thresholds

aggregate scores

In [89]:
import json
print(json.dumps(result, indent=4))

{
    "location": "13.34,74.74",
    "lat": 13.34,
    "lon": 74.74,
    "history_risk": "low",
    "weather_analysis": {
        "max_precip_mm": 0.0,
        "max_temp_c": 30.4,
        "max_wind_ms": 21.9,
        "risks_found": {
            "storm": {
                "label": "Storm Risk",
                "metric": 21.9
            }
        }
    },
    "aggregate": {
        "score": 0.25,
        "level": "Low",
        "history_score": 0.1,
        "weather_score": 0.4
    },
    "policy": "Home Insurance"
}


In [90]:
user_input_test = {
    "location": "13.34,74.74", 
    "policy": "Home Insurance",
    "history": ""
}

result = risk_assessment_engine(user_input_test)
result

{'location': '13.34,74.74',
 'lat': 13.34,
 'lon': 74.74,
 'history_risk': 'low',
 'weather_analysis': {'max_precip_mm': 0.0,
  'max_temp_c': 30.8,
  'max_wind_ms': 21.6,
  'risks_found': {'storm': {'label': 'Storm Risk', 'metric': 21.6}}},
 'aggregate': {'score': 0.25,
  'level': 'Low',
  'history_score': 0.1,
  'weather_score': 0.4},
 'policy': 'Home Insurance'}

# Use-case function: disaster_preparedness_planner (final tool)

In [91]:
def disaster_preparedness_planner(user_input, simulate_interaction_inputs=None):
    """
    Full composed function:
    - guides user (optional)
    - runs risk_assessment_engine()
    - creates preparedness workflow
    - displays information
    Returns the final plan object
    """
    # Guide user / collect input if not present
    if simulate_interaction_inputs:
        user_data = guide_user_interaction(simulate_interaction_inputs)
    else:
        user_data = user_input

    # Run risk engine
    risk_result = risk_assessment_engine(user_data)

    # Make preparedness plan from risks_found
    plan_obj = create_preparedness_workflow(risk_result["location"], risk_result["policy"], risk_result["weather_analysis"].get("risks_found", {}))

    # Add risk meta
    plan_obj["risk_summary"] = risk_result["aggregate"]
    plan_obj["weather_analysis"] = risk_result["weather_analysis"]

    # Display (or return) the plan
    display_plan(plan_obj)
    return plan_obj

In [92]:
test_user = {
    "location": "Udupi",
    "policy": "Standard Home Insurance",
    "history": "We were evacuated last month due to strong winds"
}

final_plan = disaster_preparedness_planner(test_user)
final_plan

=== Disaster Preparedness Plan ===
Location: Udupi
Policy: Standard Home Insurance
Generated at (UTC): 2025-12-11T11:56:07.236458

Plan steps:
1. Keep important documents in waterproof bags.
2. Maintain a small emergency medical kit.
3. Know the nearest shelter and evacuation route.
4. --- Storm Risk (metric: 15.3) ---
5. Secure outdoor furniture.
6. Charge your devices and power banks.
7. Stay indoors and avoid travel during storm warnings.
8. Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact.

(End of plan)


{'location': 'Udupi',
 'policy': 'Standard Home Insurance',
 'plan': ['Keep important documents in waterproof bags.',
  'Maintain a small emergency medical kit.',
  'Know the nearest shelter and evacuation route.',
  '--- Storm Risk (metric: 15.3) ---',
  'Secure outdoor furniture.',
  'Charge your devices and power banks.',
  'Stay indoors and avoid travel during storm warnings.',
  "Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact."],
 'generated_at': '2025-12-11T11:56:07.236458',
 'risk_summary': {'score': 0.65,
  'level': 'Medium',
  'history_score': 0.9,
  'weather_score': 0.4},
 'weather_analysis': {'max_precip_mm': 0.0,
  'max_temp_c': 35.3,
  'max_wind_ms': 15.3,
  'risks_found': {'storm': {'label': 'Storm Risk', 'metric': 15.3}}}}

In [93]:
sample_input = {
    "location": "Udupi",     # city name (geocoding will be used)
    "policy": "Standard Home Insurance",
    "history": "Minor flooding near my house 2 years ago; streets flooded once."
}

# Run the planner (this will call two free HTTP APIs: geocode.maps.co and open-meteo)
plan = disaster_preparedness_planner(sample_input)
# Optionally inspect plan as JSON
pprint(plan)

=== Disaster Preparedness Plan ===
Location: Udupi
Policy: Standard Home Insurance
Generated at (UTC): 2025-12-11T11:56:36.838860

Plan steps:
1. Keep important documents in waterproof bags.
2. Maintain a small emergency medical kit.
3. Know the nearest shelter and evacuation route.
4. --- Storm Risk (metric: 15.3) ---
5. Secure outdoor furniture.
6. Charge your devices and power banks.
7. Stay indoors and avoid travel during storm warnings.
8. Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact.

(End of plan)
{'generated_at': '2025-12-11T11:56:36.838860',
 'location': 'Udupi',
 'plan': ['Keep important documents in waterproof bags.',
          'Maintain a small emergency medical kit.',
          'Know the nearest shelter and evacuation route.',
          '--- Storm Risk (metric: 15.3) ---',
          'Secure outdoor furniture.',
          'Charge your devices and power banks.',
          'S

# SAVE 

In [95]:
final_plan = disaster_preparedness_planner(test_user)
final_plan

# === Save the output JSON ===
import json
import os

output_path = r"C:\Users\Gouthum\Downloads\Assisto Technologies Inc\example"
os.makedirs(output_path, exist_ok=True)

with open(os.path.join(output_path, "preparedness_plan_output.json"), "w") as f:
    json.dump(final_plan, f, indent=2)

print("preparedness_plan_output.json has been saved successfully!")

=== Disaster Preparedness Plan ===
Location: Udupi
Policy: Standard Home Insurance
Generated at (UTC): 2025-12-11T12:01:52.343932

Plan steps:
1. Keep important documents in waterproof bags.
2. Maintain a small emergency medical kit.
3. Know the nearest shelter and evacuation route.
4. --- Storm Risk (metric: 15.3) ---
5. Secure outdoor furniture.
6. Charge your devices and power banks.
7. Stay indoors and avoid travel during storm warnings.
8. Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact.

(End of plan)
preparedness_plan_output.json has been saved successfully!


# LLM

In [52]:
!pip install groq

Defaulting to user installation because normal site-packages is not writeable


In [68]:
client = Groq()

In [69]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [70]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [61]:
from groq import Groq

client = Groq(api_key="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")  

# LLM Function: Turns plan into friendly message

In [81]:
from groq import Groq

# Initialize Groq client
client = Groq(api_key="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")   # Replace with your actual key

def llm_generate_plan(plan_obj):
    """
    Uses Groq Llama 3.3 70B Versatile to generate a friendly,
    natural-language disaster preparedness advisory.
    """
    prompt = f"""
You are a Disaster Preparedness Advisor AI.

Convert the following structured plan into a clear, human-friendly advisory.

RISK SUMMARY:
{plan_obj['risk_summary']}

WEATHER ANALYSIS:
{plan_obj['weather_analysis']}

PLAN STEPS:
{plan_obj['plan']}

Write only the final advisory message in well-structured paragraphs.
    """

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
    )

    # IMPORTANT: ✔ Correct content access method
    return response.choices[0].message.content


# RUN LLM OUTPUT

In [83]:
plan_obj = disaster_preparedness_planner({
    "location": "Bengaluru",
    "policy": "Standard Home Insurance",
    "history": "There were strong winds last year"
})

=== Disaster Preparedness Plan ===
Location: Bengaluru
Policy: Standard Home Insurance
Generated at (UTC): 2025-12-11T08:08:33.719533

Plan steps:
1. Keep important documents in waterproof bags.
2. Maintain a small emergency medical kit.
3. Know the nearest shelter and evacuation route.
4. No immediate weather-based risks detected. Maintain general preparedness.
5. Policy note: Your recorded policy is 'Standard Home Insurance'. Check policy coverages for relevant damages (flood/storm) and save insurer contact.

(End of plan)


In [84]:
human_readable_output = llm_generate_plan(plan_obj)
print(human_readable_output)

**Disaster Preparedness Advisory**

Based on our current risk assessment, the overall risk level is classified as Low, with a score of 0.25. This indicates that the likelihood of a disaster occurring in your area is relatively low. However, it is essential to maintain general preparedness to ensure your safety and well-being.

Our weather analysis indicates that there are no immediate weather-based risks detected. The forecast shows a maximum temperature of 26.2°C, moderate winds of 14.8 m/s, and no precipitation expected. This suggests a relatively calm and stable weather condition. Nevertheless, it is crucial to be prepared for any unexpected events.

To ensure your preparedness, we recommend taking a few simple steps. Keep your important documents in waterproof bags to protect them from potential damage. Maintain a small emergency medical kit with essential supplies, such as bandages, antiseptic wipes, and any prescription medications. Additionally, familiarize yourself with the nea